In [4]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/RawData.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 15
n_encoder_h_2 = 12
n_encoder_h_3 = 9


#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 12
n_decoder_h_2 = 15
n_decoder_h_3 = 19



# Parameters
learning_rate = 0.01

#batch_size = 7
display_step = 1

total_batch=7

training_epochs = 7000
#we store the variables here

def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [19, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

       
        with tf.variable_scope("code"):
            output = layer(h_2, [n_encoder_h_2, n_code], [n_code], phase_train)

    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)


        with tf.variable_scope("output"):
            output = layer(h_2, [n_decoder_h_2, 19], [19], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '9'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 19]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive100=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    #val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/auto19-15-12-9/tmp/model.ckpt')
                        togive4=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,19)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive4.append(output_any_image)
                            
                        togive100=togive4
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)


we begin
Epoch: 0001 cost = 2.289329699
Validation Loss: 2.2801063
Epoch: 0002 cost = 2.269012383
Validation Loss: 2.0984476
Epoch: 0003 cost = 2.248608794
Validation Loss: 2.057244
Epoch: 0004 cost = 2.227133547
Validation Loss: 1.9757622
Epoch: 0005 cost = 2.203443732
Validation Loss: 1.9098324
Epoch: 0006 cost = 2.173669440
Validation Loss: 1.7587619
Epoch: 0007 cost = 2.138110535
Validation Loss: 1.6666254
Epoch: 0008 cost = 2.106282030
Validation Loss: 1.53381
Epoch: 0009 cost = 2.082834244
Validation Loss: 1.461424
Epoch: 0010 cost = 2.063675267
Validation Loss: 1.5022717
Epoch: 0011 cost = 2.047402348
Validation Loss: 1.8543032
Epoch: 0012 cost = 2.032686302
Validation Loss: 1.8993511
Epoch: 0013 cost = 2.018666404
Validation Loss: 1.8737859
Epoch: 0014 cost = 2.005048071
Validation Loss: 1.8214786
Epoch: 0015 cost = 1.991114429
Validation Loss: 1.6960526
Epoch: 0016 cost = 1.976401363
Validation Loss: 1.5535936
Epoch: 0017 cost = 1.961231760
Validation Loss: 1.8243077
Epoch: 00

Epoch: 0148 cost = 0.981093637
Validation Loss: 1.0667142
Epoch: 0149 cost = 0.976387901
Validation Loss: 0.8954428
Epoch: 0150 cost = 0.971711738
Validation Loss: 0.8187514
Epoch: 0151 cost = 0.967064721
Validation Loss: 0.77525663
Epoch: 0152 cost = 0.962446758
Validation Loss: 0.65261024
Epoch: 0153 cost = 0.957857856
Validation Loss: 0.8695637
Epoch: 0154 cost = 0.953297428
Validation Loss: 0.9950866
Epoch: 0155 cost = 0.948765942
Validation Loss: 1.0666445
Epoch: 0156 cost = 0.944262726
Validation Loss: 1.3073258
Epoch: 0157 cost = 0.939787967
Validation Loss: 1.0721354
Epoch: 0158 cost = 0.935340941
Validation Loss: 0.968023
Epoch: 0159 cost = 0.930922210
Validation Loss: 0.960693
Epoch: 0160 cost = 0.926531179
Validation Loss: 0.74549204
Epoch: 0161 cost = 0.922167948
Validation Loss: 0.7299965
Epoch: 0162 cost = 0.917832085
Validation Loss: 0.76332754
Epoch: 0163 cost = 0.913523631
Validation Loss: 0.86334926
Epoch: 0164 cost = 0.909242409
Validation Loss: 0.7247629
Epoch: 0165

Epoch: 0293 cost = 0.532390569
Validation Loss: 0.4731952
Epoch: 0294 cost = 0.530491880
Validation Loss: 0.5176683
Epoch: 0295 cost = 0.528605648
Validation Loss: 0.52785605
Epoch: 0296 cost = 0.526731823
Validation Loss: 0.4103092
Epoch: 0297 cost = 0.524870958
Validation Loss: 0.42317748
Epoch: 0298 cost = 0.523022907
Validation Loss: 0.46548587
Epoch: 0299 cost = 0.521188080
Validation Loss: 0.4996142
Epoch: 0300 cost = 0.519366596
Validation Loss: 0.53976625
Epoch: 0301 cost = 0.517559337
Validation Loss: 0.68393326
Epoch: 0302 cost = 0.515766710
Validation Loss: 0.8058183
Epoch: 0303 cost = 0.513989704
Validation Loss: 0.58088267
Epoch: 0304 cost = 0.512229379
Validation Loss: 0.671664
Epoch: 0305 cost = 0.510487654
Validation Loss: 0.53304565
Epoch: 0306 cost = 0.508767098
Validation Loss: 0.49071762
Epoch: 0307 cost = 0.507071491
Validation Loss: 0.6878856
Epoch: 0308 cost = 0.505406865
Validation Loss: 0.8842628
Epoch: 0309 cost = 0.503782800
Validation Loss: 1.0324972
Epoch: 

Epoch: 0435 cost = 0.347206771
Validation Loss: 0.35327598
Epoch: 0436 cost = 0.346505178
Validation Loss: 0.31073183
Epoch: 0437 cost = 0.345911724
Validation Loss: 0.2930061
Epoch: 0438 cost = 0.345488633
Validation Loss: 0.26954746
Epoch: 0439 cost = 0.345345267
Validation Loss: 0.28442693
Epoch: 0440 cost = 0.345655773
Validation Loss: 0.45096293
Epoch: 0441 cost = 0.346501052
Validation Loss: 0.4050154
Epoch: 0442 cost = 0.347027153
Validation Loss: 0.41501528
Epoch: 0443 cost = 0.345641115
Validation Loss: 0.2849808
Epoch: 0444 cost = 0.343546139
Validation Loss: 0.3212418
Epoch: 0445 cost = 0.340858149
Validation Loss: 0.39362624
Epoch: 0446 cost = 0.340118902
Validation Loss: 0.35669497
Epoch: 0447 cost = 0.338646442
Validation Loss: 0.34097016
Epoch: 0448 cost = 0.336883690
Validation Loss: 0.30951914
Epoch: 0449 cost = 0.335742925
Validation Loss: 0.31878367
Epoch: 0450 cost = 0.334739212
Validation Loss: 0.3005262
Epoch: 0451 cost = 0.333995934
Validation Loss: 0.29627585
Ep

Validation Loss: 0.25795352
Epoch: 0581 cost = 0.259728453
Validation Loss: 0.27094728
Epoch: 0582 cost = 0.259323769
Validation Loss: 0.294994
Epoch: 0583 cost = 0.258922196
Validation Loss: 0.32027268
Epoch: 0584 cost = 0.258536941
Validation Loss: 0.30807075
Epoch: 0585 cost = 0.258172987
Validation Loss: 0.31096324
Epoch: 0586 cost = 0.257821304
Validation Loss: 0.303726
Epoch: 0587 cost = 0.257464707
Validation Loss: 0.25286016
Epoch: 0588 cost = 0.257087510
Validation Loss: 0.25632846
Epoch: 0589 cost = 0.256686883
Validation Loss: 0.26342297
Epoch: 0590 cost = 0.256276441
Validation Loss: 0.25977802
Epoch: 0591 cost = 0.255881716
Validation Loss: 0.25635472
Epoch: 0592 cost = 0.255528516
Validation Loss: 0.28021955
Epoch: 0593 cost = 0.255231444
Validation Loss: 0.2864485
Epoch: 0594 cost = 0.254986135
Validation Loss: 0.29644307
Epoch: 0595 cost = 0.254771086
Validation Loss: 0.34362292
Epoch: 0596 cost = 0.254552622
Validation Loss: 0.4522942
Epoch: 0597 cost = 0.254295611
Val

...

Epoch: 6374 cost = 0.042625625
Validation Loss: 0.06662755
Epoch: 6375 cost = 0.042654780
Validation Loss: 0.066172026
Epoch: 6376 cost = 0.042645022
Validation Loss: 0.06580567
Epoch: 6377 cost = 0.042642666
Validation Loss: 0.06759413
Epoch: 6378 cost = 0.042637334
Validation Loss: 0.06683711
Epoch: 6379 cost = 0.042619097
Validation Loss: 0.061686464
Epoch: 6380 cost = 0.042627473
Validation Loss: 0.06422369
Epoch: 6381 cost = 0.042600103
Validation Loss: 0.066226065
Epoch: 6382 cost = 0.042618042
Validation Loss: 0.06425827
Epoch: 6383 cost = 0.042634207
Validation Loss: 0.062381607
Epoch: 6384 cost = 0.042626718
Validation Loss: 0.064380504
Epoch: 6385 cost = 0.042654463
Validation Loss: 0.061893646
Epoch: 6386 cost = 0.042641583
Validation Loss: 0.060962066
Epoch: 6387 cost = 0.042639629
Validation Loss: 0.063140824
Epoch: 6388 cost = 0.042629594
Validation Loss: 0.06464772
Epoch: 6389 cost = 0.042613001
Validation Loss: 0.061031964
Epoch: 6390 cost = 0.042622740
Validation Loss:

Epoch: 6518 cost = 0.042568003
Validation Loss: 0.06831333
Epoch: 6519 cost = 0.042556333
Validation Loss: 0.06758878
Epoch: 6520 cost = 0.042548797
Validation Loss: 0.065897755
Epoch: 6521 cost = 0.042562509
Validation Loss: 0.06716505
Epoch: 6522 cost = 0.042540965
Validation Loss: 0.06738308
Epoch: 6523 cost = 0.042543955
Validation Loss: 0.064386934
Epoch: 6524 cost = 0.042564641
Validation Loss: 0.067045115
Epoch: 6525 cost = 0.042553341
Validation Loss: 0.06684862
Epoch: 6526 cost = 0.042594750
Validation Loss: 0.064778365
Epoch: 6527 cost = 0.042617691
Validation Loss: 0.059823137
Epoch: 6528 cost = 0.042637232
Validation Loss: 0.06395661
Epoch: 6529 cost = 0.042699283
Validation Loss: 0.06275774
Epoch: 6530 cost = 0.042665427
Validation Loss: 0.06237421
Epoch: 6531 cost = 0.042649467
Validation Loss: 0.065739445
Epoch: 6532 cost = 0.042680876
Validation Loss: 0.060655035
Epoch: 6533 cost = 0.042642469
Validation Loss: 0.05926735
Epoch: 6534 cost = 0.042621035
Validation Loss: 0

Epoch: 6659 cost = 0.042625470
Validation Loss: 0.06050648
Epoch: 6660 cost = 0.042708165
Validation Loss: 0.061379205
Epoch: 6661 cost = 0.042664350
Validation Loss: 0.06792534
Epoch: 6662 cost = 0.042626285
Validation Loss: 0.068128295
Epoch: 6663 cost = 0.042708541
Validation Loss: 0.06877919
Epoch: 6664 cost = 0.042664249
Validation Loss: 0.06887253
Epoch: 6665 cost = 0.042625921
Validation Loss: 0.06773849
Epoch: 6666 cost = 0.042706279
Validation Loss: 0.06392867
Epoch: 6667 cost = 0.042662963
Validation Loss: 0.06987629
Epoch: 6668 cost = 0.042623739
Validation Loss: 0.06749348
Epoch: 6669 cost = 0.042700498
Validation Loss: 0.062216666
Epoch: 6670 cost = 0.042660040
Validation Loss: 0.06859521
Epoch: 6671 cost = 0.042619296
Validation Loss: 0.06665752
Epoch: 6672 cost = 0.042690815
Validation Loss: 0.06429371
Epoch: 6673 cost = 0.042654651
Validation Loss: 0.06828459
Epoch: 6674 cost = 0.042611473
Validation Loss: 0.0677586
Epoch: 6675 cost = 0.042675849
Validation Loss: 0.0619

Epoch: 6801 cost = 0.042493285
Validation Loss: 0.064559415
Epoch: 6802 cost = 0.042513934
Validation Loss: 0.0634988
Epoch: 6803 cost = 0.042518966
Validation Loss: 0.06763431
Epoch: 6804 cost = 0.042493820
Validation Loss: 0.06466831
Epoch: 6805 cost = 0.042515153
Validation Loss: 0.06457655
Epoch: 6806 cost = 0.042518690
Validation Loss: 0.0696262
Epoch: 6807 cost = 0.042493877
Validation Loss: 0.06787914
Epoch: 6808 cost = 0.042515489
Validation Loss: 0.06843915
Epoch: 6809 cost = 0.042517785
Validation Loss: 0.0690909
Epoch: 6810 cost = 0.042493351
Validation Loss: 0.06702516
Epoch: 6811 cost = 0.042515146
Validation Loss: 0.066851735
Epoch: 6812 cost = 0.042516538
Validation Loss: 0.06759853
Epoch: 6813 cost = 0.042492584
Validation Loss: 0.06567858
Epoch: 6814 cost = 0.042514638
Validation Loss: 0.06360269
Epoch: 6815 cost = 0.042515190
Validation Loss: 0.06780495
Epoch: 6816 cost = 0.042491839
Validation Loss: 0.0645461
Epoch: 6817 cost = 0.042513951
Validation Loss: 0.06695133

Validation Loss: 0.06660005
Epoch: 6944 cost = 0.042474307
Validation Loss: 0.06545848
Epoch: 6945 cost = 0.042442495
Validation Loss: 0.06765731
Epoch: 6946 cost = 0.042441099
Validation Loss: 0.06364497
Epoch: 6947 cost = 0.042465766
Validation Loss: 0.066309825
Epoch: 6948 cost = 0.042449663
Validation Loss: 0.06832126
Epoch: 6949 cost = 0.042455432
Validation Loss: 0.062111612
Epoch: 6950 cost = 0.042487861
Validation Loss: 0.060439058
Epoch: 6951 cost = 0.042457557
Validation Loss: 0.06723329
Epoch: 6952 cost = 0.042443275
Validation Loss: 0.055070233
Epoch: 6953 cost = 0.042445133
Validation Loss: 0.055822793
Epoch: 6954 cost = 0.042423205
Validation Loss: 0.06784442
Epoch: 6955 cost = 0.042426902
Validation Loss: 0.065419234
Epoch: 6956 cost = 0.042453187
Validation Loss: 0.06714614
Epoch: 6957 cost = 0.042441930
Validation Loss: 0.06787706
Epoch: 6958 cost = 0.042442345
Validation Loss: 0.06648062
Epoch: 6959 cost = 0.042462845
Validation Loss: 0.06367383
Epoch: 6960 cost = 0.0

In [5]:
import csv
with open('C:/Users/yy2895/Desktop/auto_result19-15-12-9.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive100)):
        wr.writerow(list(togive100[i][0]))

In [6]:
togive100

[array([[0.97884697, 0.77075404, 0.71253616, 0.35039315, 0.77383125,
         0.66699773, 0.43910125, 0.24961774, 0.50400877]], dtype=float32),
 array([[0.94807905, 0.5956585 , 0.7692981 , 0.29587108, 0.5544725 ,
         0.7058399 , 0.50331706, 0.43736914, 0.6234538 ]], dtype=float32),
 array([[0.91533864, 0.5770175 , 0.6479014 , 0.3442036 , 0.6646217 ,
         0.61396   , 0.48829094, 0.2806363 , 0.48187464]], dtype=float32),
 array([[0.7732914 , 0.38304874, 0.6537126 , 0.3724349 , 0.6155395 ,
         0.56891936, 0.6252464 , 0.4208655 , 0.50955904]], dtype=float32),
 array([[0.35916284, 0.2769537 , 0.4526855 , 0.56200486, 0.7978249 ,
         0.33293426, 0.75007373, 0.38417524, 0.17010938]], dtype=float32),
 array([[0.2700058 , 0.34327048, 0.66362095, 0.44445133, 0.41224277,
         0.5544071 , 0.6117215 , 0.5654323 , 0.29020548]], dtype=float32),
 array([[0.08955914, 0.21990353, 0.27804375, 0.64679205, 0.865623  ,
         0.24964607, 0.7852204 , 0.24337617, 0.11031713]], dtype=fl

In [7]:
currentmin

0.04933466